<a href="https://colab.research.google.com/github/pinodepietro/esercizi-corso-AI/blob/main/Copia_di_simulated_annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# 8-Regine con Simulated Annealing (Colab app)
# UI: Start / Stop / Step / Reset + slider per T0 (temperatura) e α (raffreddamento)
# Visualizza le mosse sulla scacchiera con ipycanvas
# ============================================================

# 0) Dipendenze e abilitazione widget manager (necessario in Colab)
import sys, subprocess
def _ensure(pkg):
    try:
        __import__(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
_ensure("ipywidgets")
_ensure("ipycanvas")

import ipywidgets as W
from ipycanvas import Canvas, hold_canvas
import numpy as np, random
from IPython.display import display
try:
    from google.colab import output as colab_output
    colab_output.enable_custom_widget_manager()
    from google.colab import output
except Exception:
    # Fuori da Colab, output.eval_js non è disponibile: definiamo un surrogato innocuo.
    class _Dummy:
        def eval_js(self, code): return None
        def register_callback(self, *a, **k): pass
    output = _Dummy()

# ============================================================
# Configurazione
# ============================================================
N = 8
CELL = 64
WIDTH = HEIGHT = N * CELL

T0_DEFAULT      = 5.0      # temperatura iniziale (slider)
ALPHA_DEFAULT   = 0.995    # raffreddamento geometrico iniziale (slider)
T_CUTOFF        = 1e-4     # soglia per considerare "raffreddato"
MAX_MOVES       = 50000    # limite di sicurezza
STEP_INTERVAL_MS= 140      # ms tra step nel timer (solo velocità visuale)

# ============================================================
# Stato Simulated Annealing
# ============================================================
state  = list(np.random.default_rng(42).integers(0, N, size=N))  # iniziale random
state0 = state.copy()

moves = 0
last_move = None          # (col, from_row, to_row) se accettata
last_dE = None            # Δh dell'ultima proposta
last_p = None             # prob. usata per accettare mossa peggiorativa
last_uphill = False       # True se l'ultima mossa accettata era peggiorativa
stuck = False             # si è raffreddato / ha raggiunto limiti

T0 = T0_DEFAULT
alpha = ALPHA_DEFAULT
T  = T0

# ============================================================
# Funzioni problema 8-regine
# ============================================================
def conflicts(board):
    """Numero di coppie di regine in conflitto (stessa riga o diagonali)."""
    c = 0
    for i in range(N):
        for j in range(i+1, N):
            if board[i] == board[j] or abs(board[i]-board[j]) == abs(i-j):
                c += 1
    return c

def neighbor(board, rng):
    """Muove una regina: sceglie una colonna e una nuova riga diversa."""
    b = board.copy()
    col = rng.integers(0, N)
    r0  = b[col]
    new_r = rng.integers(0, N-1)
    if new_r >= r0: new_r += 1
    b[col] = new_r
    return b, (col, r0, new_r)

# ============================================================
# Un passo di SA
# ============================================================
rng = np.random.default_rng(123)

def sa_step():
    """Ritorna True se proseguire, False se fermare (soluzione/raffreddato/limiti)."""
    global state, moves, last_move, last_dE, last_p, last_uphill, stuck, T

    cur_h = conflicts(state)
    if cur_h == 0:
        last_move = None; last_dE = None; last_p = None; last_uphill = False
        stuck = False
        return False

    cand, mv = neighbor(state, rng)
    cand_h = conflicts(cand)
    dE = cand_h - cur_h
    last_dE = int(dE)
    last_p = None
    last_uphill = False

    if dE <= 0:
        accept = True
    else:
        p = float(np.exp(-dE / max(1e-12, T)))
        last_p = p
        accept = (random.random() < p)
        if accept: last_uphill = True

    if accept:
        state = cand
        last_move = mv
        moves += 1
    else:
        last_move = None  # evidenziamo solo mosse accettate

    # Raffreddamento (α ora è regolabile da slider)
    T = T * alpha

    # Condizioni di stop "robuste"
    if conflicts(state) == 0:
        stuck = False
        return False
    if T < T_CUTOFF or moves >= MAX_MOVES:
        stuck = True
        return False
    return True

# ============================================================
# Rendering scacchiera (ipycanvas)
# ============================================================
canvas = Canvas(width=WIDTH, height=HEIGHT)

def draw_board():
    with hold_canvas(canvas):
        canvas.clear()
        # scacchiera a scacchi
        for x in range(N):
            for y in range(N):
                light = ((x + y) % 2 == 0)
                canvas.fill_style = "#f0d9b5" if light else "#b58863"
                canvas.fill_rect(x*CELL, (N-1-y)*CELL, CELL, CELL)

        # evidenzia ultima mossa accettata
        if last_move is not None:
            c, r_from, r_to = last_move
            canvas.fill_style = "rgba(255, 193, 7, 0.35)"  # partenza (giallo)
            canvas.fill_rect(c*CELL, (N-1-r_from)*CELL, CELL, CELL)
            canvas.fill_style = "rgba(25, 135, 84, 0.35)"  # arrivo (verde)
            canvas.fill_rect(c*CELL, (N-1-r_to)*CELL, CELL, CELL)

        # regine
        for c, r in enumerate(state):
            cx = c*CELL + CELL/2
            cy = (N-1-r)*CELL + CELL/2
            canvas.fill_style = "#1f2937"
            canvas.fill_circle(cx, cy, CELL*0.26)
            # piccola "corona"
            canvas.fill_style = "#111827"
            canvas.fill_polygon([
                (cx - CELL*0.18, cy + CELL*0.05),
                (cx + CELL*0.18, cy + CELL*0.05),
                (cx,             cy + CELL*0.22),
            ])

        # HUD con metriche
        h = conflicts(state)
        msg = f"h = {h}   mosse = {moves}   T = {T:.4f}   α = {alpha:.4f}"
        if last_dE is not None:
            msg += f"   Δh={last_dE:+d}"
            if last_uphill and last_p is not None:
                msg += f" (acc. p={last_p:.2f})"
        if h == 0:
            msg += "   ✅ soluzione!"
        elif stuck:
            msg += "   ⛔ raffreddato / max mosse"
        canvas.fill_style = "#111827"
        canvas.font = "14px sans-serif"
        canvas.fill_text(msg, 8, 18)

# click: imposta manualmente la regina nella colonna cliccata (setup iniziale)
def on_click(x, y):
    global state, state0, moves, last_move, last_dE, last_p, last_uphill, stuck, T
    c = int(x // CELL)
    r = N - 1 - int(y // CELL)
    if 0 <= c < N and 0 <= r < N:
        state[c] = r
        state0 = state.copy()
        moves = 0
        last_move = None; last_dE = None; last_p = None; last_uphill = False
        stuck = False
        T = T0  # riparte dalla T0 corrente
        draw_board()
canvas.on_mouse_down(on_click)

# ============================================================
# UI: pulsanti + slider T0 + slider alpha
# ============================================================
btn_start = W.Button(description="Start", button_style="success")
btn_stop  = W.Button(description="Stop",  button_style="warning")
btn_step  = W.Button(description="Step",  button_style="primary")
btn_reset = W.Button(description="Reset", button_style="info")
slider_T0 = W.FloatSlider(description="T0 (temperatura)", value=T0_DEFAULT, min=0.1, max=10.0, step=0.1, readout_format=".1f")
slider_al = W.FloatSlider(description="α (raffreddamento)", value=ALPHA_DEFAULT, min=0.90, max=0.9999, step=0.0005, readout_format=".4f")

# callback Python che viene richiamata dal timer JS
def _py_tick():
    cont = sa_step()
    draw_board()
    return '0' if cont else '1'

# registra la callback invocabile da JS → Python
try:
    output.register_callback('sa_tick', lambda: _py_tick())
except Exception:
    pass  # fuori da Colab, il timer JS non verrà usato

def on_start(_):
    # stop di un eventuale timer precedente
    try:
        output.eval_js("if (window._saTimer) { clearInterval(window._saTimer); window._saTimer = null; }")
        # avvia il timer
        output.eval_js(f"""
          (function() {{
            async function _saDoTick() {{
              const r = await google.colab.kernel.invokeFunction('sa_tick', [], {{}});
              const txt = String(r.data['text/plain'] || '');
              if (txt.includes('1')) {{
                if (window._saTimer) {{ clearInterval(window._saTimer); window._saTimer = null; }}
              }}
            }}
            window._saTimer = setInterval(_saDoTick, {STEP_INTERVAL_MS});
            _saDoTick();  // primo tick
          }})();
        """)
    except Exception:
        # fallback: esegue qualche step sincrono (utile fuori da Colab)
        for _ in range(200):
            if _py_tick() == '1': break

def on_stop(_):
    try:
        output.eval_js("if (window._saTimer) { clearInterval(window._saTimer); window._saTimer = null; }")
    except Exception:
        pass

def on_step(_):
    _py_tick()

def on_reset(_):
    on_stop(None)
    global state, state0, moves, last_move, last_dE, last_p, last_uphill, stuck, T
    state = state0.copy()
    moves = 0
    last_move = None; last_dE = None; last_p = None; last_uphill = False
    stuck = False
    T = T0
    draw_board()

def on_T0_change(change):
    global T0, T
    T0 = float(change["new"])
    T = T0            # applica subito la nuova T0
    draw_board()

def on_alpha_change(change):
    global alpha
    alpha = float(change["new"])   # cambia la velocità di raffreddamento (effetto dal prossimo step)
    draw_board()

btn_start.on_click(on_start)
btn_stop.on_click(on_stop)
btn_step.on_click(on_step)
btn_reset.on_click(on_reset)
slider_T0.observe(on_T0_change, names='value')
slider_al.observe(on_alpha_change, names='value')

# Layout e avvio
ui = W.HBox([btn_start, btn_stop, btn_step, btn_reset, slider_T0, slider_al])
display(ui, canvas)
draw_board()

# -------------------------------------------
# Note d'uso:
# - Clic sulla scacchiera per impostare la disposizione iniziale (colonna → riga cliccata).
# - "T0" aumenta/diminuisce l'accettazione di peggioramenti all'inizio (T viene subito aggiornata).
# - "α" regola la velocità di raffreddamento: più vicino a 1 = raffreddamento lento (più esplorazione).
# - Start/Stop/Step/Reset controllano l'animazione; Δh e p (se mossa peggiorativa accettata) compaiono nell'HUD.
# -------------------------------------------


Canvas(height=512, width=512)